# INFO 4271 - Group Project

Issued: June 11, 2024

Due: July 22, 2024

Please submit a link to your code base (ideally with a branch that does not change anymore after the submission deadline) and your 4-page report via email to carsten.eickhoff@uni-tuebingen.de by the due date. One submission per team.

---

**Authors**: Stephan Amann, Tanja Huber, Markus Potthast, Tina Truong

**Date**: 22.07.2024

In [ ]:
# python 3.10 necessary
!python --version

# requirements:
# - numpy, sentence-transformers, torch
# - spacy, asyncio, httpx, loguru, selectolax
# - pyopenssl, streamlitm, matplotlib
%pip install -U -q setuptools wheel
%pip install -q numpy sentence-transformers torch
%pip install -q fasttext-langdetect
%pip install -U -q spacy
%pip install -q asyncio httpx loguru selectolax pyopenssl 
%pip install -q streamlit
%pip install -q matplotlib 

!python -m spacy download en_core_web_sm

# Need to download index which is a large file => need to install git lfs
# using PackageCloud.io
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
# for debian based linux systems
!sudo apt-get install git-lfs
# or download tar.gz and install manually
# https://github.com/git-lfs/git-lfs/releases/download/v3.5.1/git-lfs-linux-amd64-v3.5.1.tar.gz

# 1. Web Crawling & Indexing
Crawl the web to discover **English content related to Tübingen**. The crawled content should be stored locally. If interrupted, your crawler should be able to re-start and pick up the crawling process at any time.

In [5]:
# Crawl and Scrape
from crawler import WebCrawler
crawler = WebCrawler()
await crawler.run()
# Use keyboaad interrupt (such as CTRL-C) to stop the crawler

2024-07-23 13:21:36.342 | INFO     | crawler:task_getter:176 - Limit of 10 tasks reached for frontier BOTH
2024-07-23 13:21:36.344 | INFO     | crawler:task_getter:176 - Limit of 10 tasks reached for frontier BOTH


2024-07-23 13:21:36.600 | WARNING  | crawler:fetch:120 - Fetching https://www.kyb.tuebingen.mpg.de/de/forschung/abt/bs.html failed with Client error '404 Not Found' for url 'https://www.kyb.tuebingen.mpg.de/de/forschung/abt/bs.html'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
2024-07-23 13:21:36.679 | WARNING  | crawler:fetch:125 - Retrying https://www.kyb.tuebingen.mpg.de/de/forschung/abt/bs.html after 0 attempts.
2024-07-23 13:21:36.709 | INFO     | crawler:fetch:112 - Fetched https://www.kyb.tuebingen.mpg.de/adress successfully
2024-07-23 13:21:36.712 | DEBUG    | crawler:crawl:144 - Processing https://www.kyb.tuebingen.mpg.de/adress ...
2024-07-23 13:21:36.727 | WARNING  | scraper:language_correct:92 - Language Detection: {'lang': 'en', 'score': 0.5984677076339722}
2024-07-23 13:21:36.992 | SUCCESS  | scraper:get_urls:268 - Found 38 new URLs.
2024-07-23 13:21:36.994 | INFO     | data:remove_url:190 - Frontier: Removing url https://www.ky

In [1]:
# Indexing of the processed corpus
from index import Index
index = Index()
index.index_documents()

/home/steph/miniconda3/envs/MSE/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


corpus is loaded!


Indexing documents: 100%|██████████| 5057/5057 [00:41<00:00, 121.79it/s]


Documents indexed.
Computing topic distributions...
Topic distributions computed.
Saved index, document lengths, document embeddings, and topic distributions to files.


# 2. Query Processing 
Process a textual query and return the 100 most relevant documents from your index. Please incorporate **at least one retrieval model innovation** that goes beyond BM25 or TF-IDF. Please allow for queries to be entered either individually in an interactive user interface (see also #3 below), or via a batch file containing multiple queries at once. The batch file will be formatted to have one query per line, listing the query number, and query text as tab-separated entries. An example of the batch file for the first two queries looks like this:

```
1   tübingen attractions
2   food and drinks
```

Single queries and query files are possible, use
- `tse.search("query")`
- `tse.search_from_file("queries.txt")`

In [1]:
from tue_search import TuebingenSearchEngine as TSE 
tse = TSE()

# Example query
# 1. single query
query_result = tse.search("Wilhelm Schickard") # Friedrich Miescher
# 2. Query batch from file
# query_result = tse.search_from_file("queries.txt")

# Prints the top k results. Structure is in form of (docID, score).
k = 10
for i in range(k):
    print(f"Rank {i} : {query_result[i]}")

/home/tuna/mambaforge/envs/tueSearch/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


corpus is loaded!
Rank 1: ('5025', 0.8245178797922322)
Rank 2: ('4985', 0.8017312021731078)
Rank 3: ('1035', 0.7486135043482343)
Rank 4: ('4958', 0.6343139799244908)
Rank 5: ('85', 0.6217061469320131)
Rank 6: ('1844', 0.619331792626966)
Rank 7: ('4899', 0.5971960687418162)
Rank 8: ('340', 0.5905999009024203)
Rank 9: ('2408', 0.5822676438407626)
Rank 10: ('1261', 0.5683328730293123)


# 3. Search Result Presentation
Once you have a result set, we want to return it to the searcher in two ways: a) in an interactive user interface. For this user interface, please think of **at least one innovation** that goes beyond the traditional 10-blue-links interface that most commercial search engines employ. b) as a text file used for batch performance evaluation. The text file should be formatted to produce one ranked result per line, listing the query number, rank position, document URL and relevance score as tab-separated entries. An example of the first three lines of such a text file looks like this:

```
1   1   https://www.tuebingen.de/en/3521.html   0.725
1   2   https://www.komoot.com/guide/355570/castles-in-tuebingen-district   0.671
1   3   https://www.unimuseum.uni-tuebingen.de/en/museum-at-hohentuebingen-castle   0.529
...
1   100 https://www.tuebingen.de/en/3536.html   0.178
2   1   https://www.tuebingen.de/en/3773.html   0.956
2   2   https://www.tuebingen.de/en/4456.html   0.797
...
```

In [2]:
tse.search_from_file("queries.txt") # given a file with queries, will result in a result batch called query_results.txt

Processing queries: 100%|██████████| 6/6 [00:32<00:00,  5.40s/it]

queries processed and results saved to query_results.txt


In [7]:
import subprocess

# to start 
process = subprocess.Popen(['streamlit', 'run', 'app.py'])


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.216:8501

corpus is loaded!


In [8]:
# to stop
process.terminate()

  Stopping...


# 4. Performance Evaluation 
We will evaluate the performance of our search systems on the basis of five queries. Two of them are avilable to you now for engineering purposes:
- `tübingen attractions`
- `food and drinks`

The remaining three queries will be given to you during our final session on July 23rd. Please be prepared to run your systems and produce a single result file for all five queries live in class. That means you should aim for processing times of no more than ~1 minute per query. We will ask you to send carsten.eickhoff@uni-tuebingen.de that file.

# Grading
Your final projects will be graded along the following criteria:
- 25% Code correctness and quality (to be delivered on this sheet)
- 25% Report (4 pages, PDF, explanation and justification of your design choices)
- 25% System performance (based on how well your system performs on the 5 queries relative to the other teams in terms of nDCG)
- 15% Creativity and innovativeness of your approach (in particular with respect to your search system #2 and user interface #3 innovations)
- 10% Presentation quality and clarity

# Permissible libraries
You can use any general-puprose ML and NLP libraries such as scipy, numpy, scikit-learn, spacy, nltk, but please stay away from dedicated web crawling or search engine toolkits such as scrapy, whoosh, lucene, terrier, galago and the likes. Pretrained models are fine to use as part of your system, as long as they have not been built/trained for retrieval. 
